In [ ]:
#Run this line first if you do not have xnat py installed. 
%pip install xnat
%pip install python-utils

In [ ]:
import xnat
import os 
import time

#things to edit:
username = "username"
pas = "password"
#provide the entire url to the xnat you're using:
xnat_url = "https://demo.xnat.org"
#path to the subjects folder:
pathToYourData = "/PATH/TO/SUBJECTS"
#choose a project name, if it doesn't exist one will be created for you
project_name = "projectName"

#NOTES: If the subject and subsiquent folders exist data will be merged, and any empty folders will be ignored and not uploaded. 

#DONT EDIT ANYTHING BELOW!!!
newprojectLoc = "/data/projects/"+project_name
# dataPath = os.path.abspath(pathToYourData)
dataPath = pathToYourData
session = xnat.connect(xnat_url, user=username, password=pas)

if project_name in session.projects:
    print("project exits adding subjects and data to: ", project_name)
else:
    print("Creating project ", project_name)
    session.put(newprojectLoc)
    session = xnat.connect(xnat_url, user=username, password=pas)
    while project_name not in session.projects:
        time.sleep(.5)

print("removing .ds_storefiles if they exist")
for root, dirs, files in os.walk(dataPath):
    for file in files:
        if file.endswith('.DS_Store'):
            path = os.path.join(root, file)
            os.remove(path)

print("creating subjects and uploading data")
for dirs in os.walk(dataPath):
    dirpath = dirs[0].replace("\\", "/")
    # print(dirpath)
    if "E_matrices" in dirpath:
        # print(dirpath)
        print("E matrices, uploading at project level")
        resources = session.projects[project_name].resources
        if "E_matrices" not in resources:
            print("creating E_matrices Resource")
            session.projects[project_name].create_resource("E_matrices")
            session = xnat.connect(xnat_url, user=username, password=pas)
        session.projects[project_name].resources["E_matrices"].upload_dir(directory=dirpath, method="tgz_memory")
    if "A_matrices" in dirpath:
        # print(dirpath)
        print("A matrices, uploading at project level")
        resources = session.projects[project_name].resources
        if "A_matrices" not in resources:
            print("creating A_matrices Resource")
            session.projects[project_name].create_resource("A_matrices")
            session = xnat.connect(xnat_url, user=username, password=pas)
            session.projects[project_name].resources["A_matrices"].upload_dir(directory=dirpath, method="tgz_memory")
        else:
            print("A mat aleady exists please upload manually as to avoid this time consuming process")
    if "Cortical_Meshes" in dirpath:
        # print(dirpath)
        print("Cortical Meshes, uploading at project level")
        resources = session.projects[project_name].resources
        if "Cortical_Meshes" not in resources:
            print("creating Cortical_Meshes Resource")
            session.projects[project_name].create_resource("Cortical_Meshes")
            session = xnat.connect(xnat_url, user=username, password=pas)
        session.projects[project_name].resources["Cortical_Meshes"].upload_dir(directory=dirpath, method="tgz_memory")
    if "MNI_files" in dirpath:
        # print(dirpath)
        print("MNI files, uploading at project level")
        resources = session.projects[project_name].resources
        if "MNI_files" not in resources:
            print("creating MNI_files Resource")
            session.projects[project_name].create_resource("MNI_files")
            session = xnat.connect(xnat_url, user=username, password=pas)
        session.projects[project_name].resources["MNI_files"].upload_dir(directory=dirpath, method="tgz_memory")
    if "sub-" in dirpath.split("/")[-1]:
        # print(dirpath)
        subject = dirpath.split("/")[-1]
        print("subject is ", subject)

        #if subject doesnt exist create the subject under the project
        if subject not in session.projects[project_name].subjects.keys():
            print("subject doesnt exist, creating subject ", subject)
            subjectpath = newprojectLoc+"/subjects/"+subject
            print(subjectpath)
            session.put(subjectpath)
            session = xnat.connect(xnat_url, user=username, password=pas)
            while subject not in session.projects[project_name].subjects.keys():
                time.sleep(.5)
        else:
            print("subject already exists, data will be merged")
                
    
        #create a scan data folder
        scanDataPath = newprojectLoc+"/subjects/"+subject+"/resources/scan_data"
        try: 
            session.get(scanDataPath)
        except: 
            session.post(scanDataPath)

        #create session folder under scanData 
        print("uploading ", dirpath)
        session.projects[project_name].subjects[subject].resources["scan_data"].upload_dir(directory=dirpath, method="tgz_memory")

print("Upload complete, disconnecting")
session.disconnect()